In [1]:
import os
import shutil
import requests
import ssl
import socket
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [2]:
df = pd.read_csv('/kaggle/input/website-url/keywords_urls.csv')

df.head()

,Keyword,URL
0,Ecommerce,https://www.amazon.com/
1,Ecommerce,https://www.ebay.com/
2,Ecommerce,https://www.alibaba.com/
3,Ecommerce,https://www.walmart.com/
4,Ecommerce,https://www.etsy.com/


In [3]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
def get_text(url):
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text()
        # Extract and print only the text
        cleaned_text = ' '.join(text.split())
        cleaned_text = cleaned_text[:2000]
        return cleaned_text
    else:
        error = f"Error: {response.status_code}"
        return error 

In [4]:
df['text'] = df['URL'].apply(get_text)

In [5]:
def get_info(web_url):
    parsed_url = urlparse(web_url)
    protocol = parsed_url.scheme
    
    return protocol
#     url = parsed_url.netloc
#     domain_name = url.split('.')[-1]

In [6]:
def domain_info(web_url):
    parsed_url = urlparse(web_url)
    url = parsed_url.netloc
    domain_name = url.split('.')[-1]
    
    return domain_name


In [7]:
def net_loc(web_url):
    parsed_url = urlparse(web_url)
    net_loc = parsed_url.netloc
    
    return net_loc

In [8]:
df['protocol'] = df['URL'].apply(get_info)

In [9]:
df['domain'] = df["URL"].apply(domain_info)

In [10]:
df['Net_loc'] = df['URL'].apply(net_loc)

In [11]:
text_df = df['text'].reset_index(drop=False)

In [12]:
text_df = text_df.drop(columns='index')

In [13]:
df

,Keyword,URL,text,protocol,domain,Net_loc
0,Ecommerce,https://www.amazon.com/,Amazon.com Enter the characters you see below ...,https,com,www.amazon.com
1,Ecommerce,https://www.ebay.com/,"Electronics, Cars, Fashion, Collectibles & Mor...",https,com,www.ebay.com
2,Ecommerce,https://www.alibaba.com/,"Alibaba.com: Manufacturers, Suppliers, Exporte...",https,com,www.alibaba.com
3,Ecommerce,https://www.walmart.com/,Walmart | Save Money. Live better.Skip to Main...,https,com,www.walmart.com
4,Ecommerce,https://www.etsy.com/,Error: 403,https,com,www.etsy.com
5,Ecommerce,https://www.target.com/,Target : Expect More. Pay Less.skip to main co...,https,com,www.target.com
6,Ecommerce,https://www.bestbuy.com/,Best Buy International: Select your Country - ...,https,com,www.bestbuy.com
7,Ecommerce,https://www.shopify.com/,ShopifySkip to ContentShopifySolutionsStartSta...,https,com,www.shopify.com
8,Ecommerce,https://www.flipkart.com/,Error: 500,https,com,www.flipkart.com
9,Ecommerce,https://www.rakuten.com/,Rakuten: Shop. Get Cash Back. Repeat. Take adv...,https,com,www.rakuten.com


# Now Checking the similarity measure

In [14]:
#Install Packages
!pip install -q faiss-cpu
!pip install -q sentence-transformers

In [15]:
# import library

import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-10 14:59:08.006367: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 14:59:08.006516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 14:59:08.182205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [16]:
# Similarity measure model

encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
vectors = encoder.encode(text_df.text)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
vectors.shape

(50, 384)

In [18]:
dim = vectors.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(vectors)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7e791b4e7390> >

In [19]:
web_url = 'https://www.amazon.in/'
text = get_text(web_url)
text = text[:2000]

In [20]:
print(text)

Online Shopping site in India: Shop Online for Mobiles, Books, Watches, Shoes and More - Amazon.in Skip to main content .in Delivering to Mumbai 400001 Update location All Select the department you want to search in All Categories Alexa Skills Amazon Devices Amazon Fashion Amazon Fresh Amazon Fresh Meat Amazon Pharmacy Appliances Apps & Games Audible Audiobooks Baby Beauty Books Car & Motorbike Clothing & Accessories Collectibles Computers & Accessories Deals Electronics Furniture Garden & Outdoors Gift Cards Grocery & Gourmet Foods Health & Personal Care Home & Kitchen Industrial & Scientific Jewellery Kindle Store Luggage & Bags Luxury Beauty Movies & TV Shows MP3 Music Music Musical Instruments Office Products Pet Supplies Prime Video Shoes & Handbags Software Sports, Fitness & Outdoors Subscribe & Save Tools & Home Improvement Toys & Games Under ₹500 Video Games Watches Search Amazon.in EN Hello, sign in Account & Lists Returns & Orders 0 Cart All Fresh Amazon miniTV Sell Best Sell

In [21]:
vec = encoder.encode(text)
svec = np.array(vec).reshape(1,-1)
distances, I = index.search(svec, k=6)
print(distances)
print(I)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[1.0601816 1.1414766 1.1637342 1.1912718 1.2478318 1.2798336]]
[[2 0 1 9 7 3]]


In [22]:
row_idx = I[0]
row_idx

array([2, 0, 1, 9, 7, 3])

In [23]:
selected_rows = df.iloc[row_idx]
selected_rows

,Keyword,URL,text,protocol,domain,Net_loc
2,Ecommerce,https://www.alibaba.com/,"Alibaba.com: Manufacturers, Suppliers, Exporte...",https,com,www.alibaba.com
0,Ecommerce,https://www.amazon.com/,Amazon.com Enter the characters you see below ...,https,com,www.amazon.com
1,Ecommerce,https://www.ebay.com/,"Electronics, Cars, Fashion, Collectibles & Mor...",https,com,www.ebay.com
9,Ecommerce,https://www.rakuten.com/,Rakuten: Shop. Get Cash Back. Repeat. Take adv...,https,com,www.rakuten.com
7,Ecommerce,https://www.shopify.com/,ShopifySkip to ContentShopifySolutionsStartSta...,https,com,www.shopify.com
3,Ecommerce,https://www.walmart.com/,Walmart | Save Money. Live better.Skip to Main...,https,com,www.walmart.com


In [24]:
for idx in row_idx:
    print(idx)
    print(selected_rows['Net_loc'][idx])
    print(selected_rows['domain'][idx])
    print(selected_rows['protocol'][idx])
    print(selected_rows['text'][idx][:500]+"....")
    print('----------------------------------')

2
www.alibaba.com
com
https
Alibaba.com: Manufacturers, Suppliers, Exporters & Importers from the world's largest online B2B marketplace All categoriesFeatured selectionsTrade AssuranceBuyer CentralHelp CenterGet the appBecome a supplierLearn about Alibaba.comThe leading B2B ecommerce platform for global tradeWhat are you looking for?SearchMillions of business offeringsExplore products and suppliers for your business from millions of offerings worldwide.Assured quality and transactionsEnsure production quality from verif....
----------------------------------
0
www.amazon.com
com
https
Amazon.com Enter the characters you see below Sorry, we just need to make sure you're not a robot. For best results, please make sure your browser is accepting cookies. Type the characters you see in this image: Try different image Continue shopping Conditions of Use Privacy Policy © 1996-2014, Amazon.com, Inc. or its affiliates....
----------------------------------
1
www.ebay.com
com
https
Electronic

# Ranking the website based on a protocol and domain

In [25]:
def rank_security(df):
    # Define a ranking based on protocol versions
    protocol_ranking = {
        'https': 1,
        'http': 0
    }
    
    df['Security Rank'] = df['protocol'].map(protocol_ranking)
    # Sort by Security Rank (descending)
    df = df.sort_values(by='Security Rank', ascending=False)
    return df

In [26]:
def rank_domain(df):
    # Define a ranking based on protocol versions
    domain_ranking = {
        'com': 3,
        'in': 2,
        'org':1,
        'uk': 0
        
    }
    
    df['Domain Rank'] = df['domain'].map(domain_ranking)
    # Sort by Security Rank (descending)
    df = df.sort_values(by='Domain Rank', ascending=False)
    return df

In [27]:
row_index = [44, 49, 39, 12, 13, 2, 1]
selected_row = df.iloc[row_index]
selected_row

,Keyword,URL,text,protocol,domain,Net_loc
44,Travel Booking,https://www.airbnb.co.in/?locale=en&_set_bev_o...,"Airbnb | Holiday rentals, cabins, beach houses...",https,in,www.airbnb.co.in
49,Travel Booking,https://www.orbitz.com/,Error: 429,https,com,www.orbitz.com
39,Health & Fitness,https://www.nhs.uk/live-well/exercise/,Exercise - NHS Skip to main content Search the...,https,uk,www.nhs.uk
12,Online Education,https://www.edx.org/,Build new skills. Advance your career. | edX S...,https,org,www.edx.org
13,Online Education,https://www.khanacademy.org/,"Khan Academy | Free Online Courses, Lessons & ...",https,org,www.khanacademy.org
2,Ecommerce,https://www.alibaba.com/,"Alibaba.com: Manufacturers, Suppliers, Exporte...",https,com,www.alibaba.com
1,Ecommerce,https://www.ebay.com/,"Electronics, Cars, Fashion, Collectibles & Mor...",https,com,www.ebay.com


In [28]:
rank_security(selected_row)

/tmp/ipykernel_18/3936110020.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Security Rank'] = df['protocol'].map(protocol_ranking)


,Keyword,URL,text,protocol,domain,Net_loc,Security Rank
44,Travel Booking,https://www.airbnb.co.in/?locale=en&_set_bev_o...,"Airbnb | Holiday rentals, cabins, beach houses...",https,in,www.airbnb.co.in,1
49,Travel Booking,https://www.orbitz.com/,Error: 429,https,com,www.orbitz.com,1
39,Health & Fitness,https://www.nhs.uk/live-well/exercise/,Exercise - NHS Skip to main content Search the...,https,uk,www.nhs.uk,1
12,Online Education,https://www.edx.org/,Build new skills. Advance your career. | edX S...,https,org,www.edx.org,1
13,Online Education,https://www.khanacademy.org/,"Khan Academy | Free Online Courses, Lessons & ...",https,org,www.khanacademy.org,1
2,Ecommerce,https://www.alibaba.com/,"Alibaba.com: Manufacturers, Suppliers, Exporte...",https,com,www.alibaba.com,1
1,Ecommerce,https://www.ebay.com/,"Electronics, Cars, Fashion, Collectibles & Mor...",https,com,www.ebay.com,1


In [29]:
data = rank_domain(selected_row)

/tmp/ipykernel_18/3602728933.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Domain Rank'] = df['domain'].map(domain_ranking)


In [30]:
data

,Keyword,URL,text,protocol,domain,Net_loc,Security Rank,Domain Rank
49,Travel Booking,https://www.orbitz.com/,Error: 429,https,com,www.orbitz.com,1,3
2,Ecommerce,https://www.alibaba.com/,"Alibaba.com: Manufacturers, Suppliers, Exporte...",https,com,www.alibaba.com,1,3
1,Ecommerce,https://www.ebay.com/,"Electronics, Cars, Fashion, Collectibles & Mor...",https,com,www.ebay.com,1,3
44,Travel Booking,https://www.airbnb.co.in/?locale=en&_set_bev_o...,"Airbnb | Holiday rentals, cabins, beach houses...",https,in,www.airbnb.co.in,1,2
12,Online Education,https://www.edx.org/,Build new skills. Advance your career. | edX S...,https,org,www.edx.org,1,1
13,Online Education,https://www.khanacademy.org/,"Khan Academy | Free Online Courses, Lessons & ...",https,org,www.khanacademy.org,1,1
39,Health & Fitness,https://www.nhs.uk/live-well/exercise/,Exercise - NHS Skip to main content Search the...,https,uk,www.nhs.uk,1,0


In [31]:
data.index

Index([49, 2, 1, 44, 12, 13, 39], dtype='int64')

In [32]:
for idx, row in data.iterrows():
    print(f"The Location of website on internet is: {row['Net_loc']}")
    print(f"The Domain name of website is: {row['domain']}")
    print(f"The Protocol is: {row['protocol']}")
    print(f"Text: {row['text'][:500]}....")
    print('----------------------------------')

The Location of website on internet is: www.orbitz.com
The Domain name of website is: com
The Protocol is: https
Text: Error: 429....
----------------------------------
The Location of website on internet is: www.alibaba.com
The Domain name of website is: com
The Protocol is: https
Text: Alibaba.com: Manufacturers, Suppliers, Exporters & Importers from the world's largest online B2B marketplace All categoriesFeatured selectionsTrade AssuranceBuyer CentralHelp CenterGet the appBecome a supplierLearn about Alibaba.comThe leading B2B ecommerce platform for global tradeWhat are you looking for?SearchMillions of business offeringsExplore products and suppliers for your business from millions of offerings worldwide.Assured quality and transactionsEnsure production quality from verif....
----------------------------------
The Location of website on internet is: www.ebay.com
The Domain name of website is: com
The Protocol is: https
Text: Electronics, Cars, Fashion, Collectibles & More | eBay

In [33]:
protocol_weight = 0.5
domain_weight = 0.3

def rank_domain(df):
    df['score'] = df['Security Rank'] * protocol_weight + df['Domain Rank'] * domain_weight
    # Sort by Security Rank (descending)
    df = df.sort_values(by='Domain Rank', ascending=False)
    return df
    

In [34]:
rank_domain(data)

,Keyword,URL,text,protocol,domain,Net_loc,Security Rank,Domain Rank,score
49,Travel Booking,https://www.orbitz.com/,Error: 429,https,com,www.orbitz.com,1,3,1.4
2,Ecommerce,https://www.alibaba.com/,"Alibaba.com: Manufacturers, Suppliers, Exporte...",https,com,www.alibaba.com,1,3,1.4
1,Ecommerce,https://www.ebay.com/,"Electronics, Cars, Fashion, Collectibles & Mor...",https,com,www.ebay.com,1,3,1.4
44,Travel Booking,https://www.airbnb.co.in/?locale=en&_set_bev_o...,"Airbnb | Holiday rentals, cabins, beach houses...",https,in,www.airbnb.co.in,1,2,1.1
12,Online Education,https://www.edx.org/,Build new skills. Advance your career. | edX S...,https,org,www.edx.org,1,1,0.8
13,Online Education,https://www.khanacademy.org/,"Khan Academy | Free Online Courses, Lessons & ...",https,org,www.khanacademy.org,1,1,0.8
39,Health & Fitness,https://www.nhs.uk/live-well/exercise/,Exercise - NHS Skip to main content Search the...,https,uk,www.nhs.uk,1,0,0.5


# Scoring Function 

In [35]:
# Using a scoring function that helps rank secure websites on top

def protocol_to_score(df):
    # Define a ranking based on protocol versions
    protocol_ranking = {
        'https': 1,
        'http': 0
    }
    # Define a ranking based on protocol versions
    domain_ranking = {
        'com': 3,
        'in': 2,
        'org':1,
        'uk': 0
        
    }
    
    df['Domain Rank'] = df['domain'].map(domain_ranking)
    
    df['Security Rank'] = df['protocol'].map(protocol_ranking)
    
    df['score'] = df['Security Rank'] * protocol_weight + df['Domain Rank'] * domain_weight
    # Sort by Security Rank (descending)
    df = df.sort_values(by='score', ascending=False)
    return df

In [36]:
row_index = [1, 44, 49, 39, 12, 13, 2, 0]
selected_row = df.iloc[row_index]
selected_row

,Keyword,URL,text,protocol,domain,Net_loc
1,Ecommerce,https://www.ebay.com/,"Electronics, Cars, Fashion, Collectibles & Mor...",https,com,www.ebay.com
44,Travel Booking,https://www.airbnb.co.in/?locale=en&_set_bev_o...,"Airbnb | Holiday rentals, cabins, beach houses...",https,in,www.airbnb.co.in
49,Travel Booking,https://www.orbitz.com/,Error: 429,https,com,www.orbitz.com
39,Health & Fitness,https://www.nhs.uk/live-well/exercise/,Exercise - NHS Skip to main content Search the...,https,uk,www.nhs.uk
12,Online Education,https://www.edx.org/,Build new skills. Advance your career. | edX S...,https,org,www.edx.org
13,Online Education,https://www.khanacademy.org/,"Khan Academy | Free Online Courses, Lessons & ...",https,org,www.khanacademy.org
2,Ecommerce,https://www.alibaba.com/,"Alibaba.com: Manufacturers, Suppliers, Exporte...",https,com,www.alibaba.com
0,Ecommerce,https://www.amazon.com/,Amazon.com Enter the characters you see below ...,https,com,www.amazon.com


In [37]:
protocol_to_score(selected_row)

/tmp/ipykernel_18/598336355.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Domain Rank'] = df['domain'].map(domain_ranking)
/tmp/ipykernel_18/598336355.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Security Rank'] = df['protocol'].map(protocol_ranking)
/tmp/ipykernel_18/598336355.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

,Keyword,URL,text,protocol,domain,Net_loc,Domain Rank,Security Rank,score
1,Ecommerce,https://www.ebay.com/,"Electronics, Cars, Fashion, Collectibles & Mor...",https,com,www.ebay.com,3,1,1.4
49,Travel Booking,https://www.orbitz.com/,Error: 429,https,com,www.orbitz.com,3,1,1.4
2,Ecommerce,https://www.alibaba.com/,"Alibaba.com: Manufacturers, Suppliers, Exporte...",https,com,www.alibaba.com,3,1,1.4
0,Ecommerce,https://www.amazon.com/,Amazon.com Enter the characters you see below ...,https,com,www.amazon.com,3,1,1.4
44,Travel Booking,https://www.airbnb.co.in/?locale=en&_set_bev_o...,"Airbnb | Holiday rentals, cabins, beach houses...",https,in,www.airbnb.co.in,2,1,1.1
12,Online Education,https://www.edx.org/,Build new skills. Advance your career. | edX S...,https,org,www.edx.org,1,1,0.8
13,Online Education,https://www.khanacademy.org/,"Khan Academy | Free Online Courses, Lessons & ...",https,org,www.khanacademy.org,1,1,0.8
39,Health & Fitness,https://www.nhs.uk/live-well/exercise/,Exercise - NHS Skip to main content Search the...,https,uk,www.nhs.uk,0,1,0.5
